## Get the Metadata for the 990 IRS files

In [13]:
from pymongo import MongoClient
#import our own libs
# METHOD 1
#import import_ipynb
#import IRS990libs
# METHOD 2 - This will reload without restarting notebook (debugging)
#%run IRS990libs.ipynb

In [2]:
year = 2017

In [3]:
# SET TO TRUE IF YOU WANT TO CLEAN DATABASE
DUMP_DB = True

uri = "mongodb://mongo/tweets"
client = MongoClient(uri)

print( 'List of databases in MongoDB:' )
print(client.list_database_names())

# database
db = client['irs990']

# collection
dbYear = db['%s' % year]

# if we already have documents then drop them and start
# clean on this collection
if DUMP_DB and dbYear.count_documents({}) > 0:
    print( 'Drop database and start over with archive')
    dbYear.drop()

List of databases in MongoDB:
['admin', 'config', 'irs990', 'local']


In [35]:
%run IRS990libs.ipynb

documents = classy990forms(config='validElements.conf', debug=False)
documents.year( year )
documents

IRS 990 form for year [2017]: 489013 documents

In [ ]:
for x in documents:
    
    index = "%s_%s" % ( x['EIN'], year)
        
    sys.stdout.write('Download EIN:%s\r' % index)
    sys.stdout.flush() 
    
    
    dbYear.update_one( {"_id" : index}, {"$set":x.active}, upsert=True )
